In [48]:
#Tech Task: Task 2
import pandas as pd
import numpy as np
import requests
import sqlite3
from bs4 import BeautifulSoup
from time import sleep
import re


In [3]:
#Check the maximum number of crashes across all the years - max year is 1972 with 105 crashes
crashNum = {}

for i in range(1920, 2018):
    crashNum[i] = 0
    maxCrashNum = 105
    url = "http://www.planecrashinfo.com/{}/{}-{}.htm".format(i, i, maxCrashNum)
        
    try:
        r = requests.get(url)
        if r.ok:
            print("{} has more than maxCrashNum".format(i))
        else:
            print(".", end="")
            
    #Insert delay so to avoid connection being refused (Max retries exceeded with URL)
    except requests.exceptions.ConnectionError:
            
            print("Connection refused")
            sleep(5)
        

....................................................1972 has more than maxCrashNum
.............................................

In [4]:
#Get data from crash website and put in crashPages list

crashPages = list()

for year in range(1920, 2018):
    for i in range(1,110):
        url = "http://www.planecrashinfo.com/{}/{}-{}.htm".format(year, year, i)
        try:
            r = requests.get(url)
            if not r.ok:
                break
            else:
                crashPages.append(r)
                print(".", end="")
    
        #Insert delay so to avoid connection being refused (Max retries exceeded with URL)
        except requests.exceptions.ConnectionError:
            
            print("Connection refused")
            sleep(5)
    

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [5]:
crashPages[1].ok

True

In [77]:
#Parse data in crashPages list

recordList = list()
del recordList[:] #clear list

for page in crashPages:
    soup = BeautifulSoup(page.text, 'html.parser')

    table = soup.find('table')
    rows = table.find_all('tr')
    rows = rows[1:]
    
    dataDict = {}

    for row in rows:
        td_items = row.find_all('td')
        td_label = td_items[0].text.strip()
        td_data = td_items[1].text.strip()
        dataDict[td_label] = td_data
    
    #Clean data
    try:
        aboardNum = dataDict['Aboard:'].split(" ")[0] #Just take the first number, don't worry about the passenger/crew breakup
        dataDict['Aboard:'] = int(aboardNum)
        fatalityNum = dataDict['Fatalities:'].split(" ")[0] #Just take the first number, don't worry about the passenger/crew breakup
#         print("{}, ".format(fatalityNum), end="")
        dataDict['Fatalities:'] = int(fatalityNum) 
    except:
#         print('x', end="") #corrupted data
        continue
    
    recordList.append(dataDict)

In [72]:
#Insert into pandas dataframe (crashFrame)
crashFrame.drop(crashFrame.index, inplace=True) #clear dataframe
crashFrame = pd.DataFrame(recordList)

In [73]:
crashFrame['Fatalities:'].unique()

array([  1,   5,  14,  30,  21,  19,  20,  22,  27,  23,  24,  18,   0,
         2,   4,   3,  46,  34,   7,  17,   6,  52,   8,   9,  16,  10,
        48,  15,  73,  12,  11,  50,  13,  35,  25,  29,  36,  26,  40,
        31,  28,  59,  45,  44,  39,  32,  37,  38,  53,  43,  41,  33,
        55,  80,  58,  56,  70,  87, 129,  42,  47,  66,  74, 128,  62,
        67,  79,  61,  49,  99,  64,  51,  65,  60,  68,  54,  63,  72,
        78,  83,  77,  76,  95, 111, 107, 130, 112,  81,  84,  94,  86,
        97, 101, 118,  85,  57, 121, 117, 133, 124,  98,  82, 126,  88,
       123, 155, 100, 120,  90,  92, 102, 109,  75,  93, 163,  69,  91,
       104, 115, 122, 156, 108, 174, 103, 176, 110,  89, 106, 346, 191,
       113, 188, 154,  96, 583, 131, 213, 137, 183, 271, 178, 257, 146,
       166, 301, 180, 132, 145, 119, 269, 181, 148, 329, 200, 134, 520,
       256, 167, 159, 143, 290, 259, 144, 171, 127, 223, 261, 158, 141,
       157, 116, 264, 160, 189, 230, 349, 228, 234, 196, 229, 21

In [79]:
# Total fatalities between 1920-2016

total_fatalities = crashFrame['Fatalities:'].sum()
print("Total fatalities from 1920-2016: {}".format(total_fatalities))

Total fatalities from 1920-2016: 114575


In [101]:
# Top 3 airlines with the highest rate of incidents

airline_incident_groups = crashFrame.groupby(crashFrame['Operator:'])
airline_incident_frame = airline_incident_groups.sum()
airline_incident_frame = airline_incident_frame.sort_values(by='Fatalities:', ascending=False)
top_three_worst_airlines_frame = airline_incident_frame[0:3]

print("Top 3 airlines with the highest rate of incidents between 1920-2016")
top_three_worst_airlines_frame.head()

Top 3 airlines with the highest rate of incidents between 1920-2016


,Aboard:,Fatalities:
Operator:,,
Aeroflot,11682,9158
Military - U.S. Air Force,4505,3718
Air France,2886,1743


In [ ]:
# Year with the highest incidents

airline_incident_groups = crashFrame.groupby(crashFrame['Operator:'])




In [16]:
# print(crashFrame.columns.tolist())
crashFrame.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5772 entries, 0 to 5771
Data columns (total 13 columns):
AC
        Type:    5748 non-null object
Aboard:             5772 non-null object
Date:               5772 non-null object
Fatalities:         5772 non-null object
Flight #:           1337 non-null object
Ground:             5721 non-null object
Location:           5766 non-null object
Operator:           5751 non-null object
Registration:       5418 non-null object
Route:              4272 non-null object
Summary:            5540 non-null object
Time:               3654 non-null object
cn / ln:            4562 non-null object
dtypes: object(13)
memory usage: 586.3+ KB


In [20]:
# Total fatalities between period 1920-2016 period
# Top 3 airlines with the highest rate of incidents
# Year with the highest incidents

# crashFrame['Fatalities:'].replace(np.nan, 0, inplace=True)
totalFatalities = crashFrame['Fatalities:'].astype(int).sum()
print("Total fatalities in 1920-2016 period: {}".format(totalFatalities))


ValueError: invalid literal for int() with base 10: '? \xa0 (passengers:?\xa0 crew:?)'

In [70]:
SQLITE_FILE = "crashData.db"
conn = sqlite3.connect(SQLITE_FILE) #opens sqlite and database file
myCursor = conn.cursor() #provides connection to database
crashFrame.to_sql("crashTable", conn, if_exists="replace")


/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)
